This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ]), array([229.47555409, 168.55182981, 149.11300119, ...,  52.20665295,
        45.7003374 ,   6.24221959]), OrderedDict([('R2', 0.7776427550468986), ('root_mean_squared_error', 44.35971780495661), ('mean_absolute_error', 30.59655114896355), ('rmse_over_stdev', 0.4751492652470045)]), None)
test_quad = (array([3.40489474e+02, 2.53584855e+02, 1.56515230e+02, 1.10462825e+02,
       1.85182335e+02, 3.04216098e+02, 7.15466155e+01, 6.73902194e+01,
       1.02665208e+02, 1.23826343e+02, 1.17557012e+02, 1.77346326e+02,
       1.30710796e+02, 1.72420203e+02, 1.44746721e+02, 1.46059829e+02,
       1.22487016e+02, 3.21795003e+02, 1.66699389e+02, 1.78669886e+02,
       1.42483393e+02, 1.25776102e+02, 1.37316410e+02, 3.69780134e+02,
       3.19706165e+02, 5.49497902e+01, 5.96792027e+01, 1.19657588e+02,
       2.09882629e+02, 8.09403347e+01, 1.87519625e+02, 3.87490899e+01,
       2.35011420e+01, 0.00000000e+00, 1.21571007e+02, 7.66050637e+01,
       1.83350687e+02, 1.63185749e+02, 1.20214816e+02, 2.93743989e+02,
       2.09869299e+02, 7.39283454e+01, 1.22389663e+02, 1.75379626e+02,
       2.01817565e+02, 2.74349274e+02, 1.34180693e+02, 1.98061859e+02,
       2.37521275e+02, 2.78307782e+02, 1.63297865e+02, 1.79258723e+02,
       3.47342257e+01, 7.58782445e+01, 4.16012879e+01, 9.79676617e+00,
       5.61529988e+01, 7.48766194e+00, 3.63631922e+01, 1.41664751e+02,
       7.38023139e+01, 1.97421378e+02, 2.61724803e+02, 4.00299318e+01,
       1.94608833e+01, 7.45861041e+01, 1.49611167e+02, 0.00000000e+00,
       3.07830539e+02, 2.26412639e+01, 9.68586984e+01, 7.47720530e+01,
       0.00000000e+00, 2.12302521e+02, 2.89712978e+02, 1.45197832e+02,
       5.00254523e+01, 5.95521104e+01, 0.00000000e+00, 0.00000000e+00,
       4.70181371e+01, 6.21679659e+01, 0.00000000e+00, 2.03972667e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.51430137e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.53695907e+01,
       8.59609736e+01, 8.92356506e+01, 1.73373103e+01, 1.18479708e+01,
       3.49408401e+01, 6.70851015e+01, 3.27899307e+01, 3.81461657e+01,
       0.00000000e+00, 6.50312714e+01, 2.67022126e+01, 4.31252154e+01,
       3.92264457e+01, 1.62308582e+02, 2.32209209e+02, 1.81910775e+02,
       1.61615856e+02, 1.89644510e+02, 1.55792869e+02, 1.94268537e+02,
       8.64067635e+01, 1.28935952e+02, 1.40944441e+01, 1.19314875e+01,
       0.00000000e+00, 0.00000000e+00, 5.05625634e+00, 9.52352009e+00,
       1.38046084e+01, 5.26296357e+01, 3.41493981e+01, 4.23248061e+01,
       7.37896699e+00, 1.41470544e+02, 2.39414173e+01, 2.21842737e+01,
       6.53557069e+01, 9.55205228e+01, 2.08989075e+02, 5.38520339e+01,
       7.96465197e+01, 4.54332140e+01, 4.76192799e+01, 1.33902387e+01,
       4.85711869e+01, 1.57595600e+02, 9.39602340e+01, 8.04591080e+01,
       6.52948104e+01, 8.51942082e+01, 7.58854939e+01, 0.00000000e+00,
       1.29323672e+02, 1.37126059e+02, 1.93409720e+01, 4.70862505e+01,
       4.36717688e+00, 2.98368831e+02, 2.32454011e+02, 8.39171957e+01,
       0.00000000e+00, 1.85524542e+02, 2.07515714e+02, 4.73693459e+01,
       2.02664553e+02, 3.21989575e+02, 1.28448866e+02, 7.31341625e+01,
       2.53031123e+01, 4.22355880e+01, 4.08070147e+01, 2.93008151e+01,
       2.24196124e+01, 0.00000000e+00, 1.40097275e+00, 0.00000000e+00,
       5.44675418e+01, 0.00000000e+00, 2.15174731e+01, 1.84401941e+01,
       5.88853197e+01, 5.12388154e+01, 0.00000000e+00, 4.55261914e+01,
       0.00000000e+00, 4.45494170e+01, 2.04099993e+02, 9.00043474e+01,
       6.77439450e+01, 7.67577252e+01, 5.65407126e+01, 2.62753383e+01,
       5.83323071e+01, 7.68321235e+01, 9.70888906e+01, 5.13090164e+01,
       0.00000000e+00, 5.33902792e+01, 6.08628852e+01, 3.96932977e+01,
       4.28158804e+01, 0.00000000e+00, 4.43244216e+01, 0.00000000e+00,
       0.00000000e+00, 1.89559087e+01, 9.43019059e+01, 8.10394982e+01,
       3.61816245e+01, 2.64912691e+01, 2.44136176e+01, 3.11645320e+01,
       4.29040665e+01, 8.22953902e+01, 1.40515010e+02, 1.10716943e+02,
       1.01745118e+01, 1.92994335e+01, 5.29102783e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.89092871e+01, 8.06894184e+01,
       1.89735033e+01, 8.63070905e+00, 0.00000000e+00, 0.00000000e+00,
       3.69821609e+00, 5.40607889e+01, 0.00000000e+00, 1.03743969e+02,
       1.59117705e+02, 2.02269808e+02, 1.58037965e+02, 1.86400275e+02,
       3.67138367e+02, 2.76006928e+02, 8.15746890e+01, 1.29607780e+02,
       1.24477908e+02, 1.30390352e+02, 1.03756755e+02, 1.55694637e+02,
       2.59886084e+02, 1.70627481e+02, 2.85344428e+02, 2.20161511e+02,
       2.18442616e+02, 1.00864883e+02, 1.33954726e+02, 1.78586168e+02,
       8.30462436e+01, 2.10335728e+02, 1.18248560e+02, 2.51196620e+02,
       3.51817502e+02, 1.77346156e+02, 1.91512363e+02, 3.40306363e+02,
       3.40437655e+02, 2.33810236e+02, 5.98511381e+01, 7.69784095e+01,
       9.77091114e+01, 2.96095913e+02, 2.06610134e+02, 1.98592674e+02,
       8.84773614e+01, 9.44335089e+01, 1.00763646e+02, 1.35408933e+02,
       9.48376021e+01, 1.97309000e+02, 1.16278718e+02, 2.16454613e+02,
       1.60233988e+02, 2.59086381e+02, 2.59216571e+02, 2.45488977e+02,
       1.86717789e+02, 3.28541723e+02, 3.49605405e+02, 7.10448978e+01,
       7.89063338e+01, 6.56442278e+01, 1.18314107e+02, 5.46384029e+01,
       1.33883367e+02, 1.32392433e+02, 5.31282447e+01, 1.60434807e+02,
       6.92085003e+01, 1.65456460e+02, 1.80035168e+02, 8.31507296e+01,
       2.87755496e+02, 2.71906598e+02, 9.56831956e+02, 9.50227906e+02,
       1.28639131e+02, 1.20158300e+02, 8.42083837e+01, 8.46282081e+01,
       1.94691060e+02, 1.05836151e+02, 8.28280997e+01, 1.08312182e+02,
       1.78159873e+02, 1.22028582e+02, 1.89420372e+02, 9.04495542e+01,
       8.35342519e+01, 6.89800768e+01, 4.68826690e+01, 9.94733566e+00,
       9.31208466e+01, 4.96708469e+01, 7.19683460e+01, 4.18854297e+01,
       6.88773537e+01, 1.25171931e+02, 7.21745594e+01, 1.82785945e+01,
       4.85930390e+01, 4.88429076e+01, 1.61563184e+02, 1.74909069e+02,
       1.97176332e+02, 1.98013501e+02, 1.85414463e+02, 1.33312148e+02,
       1.54672899e+02, 1.15407866e+02, 1.85543114e+02, 3.32804978e+01,
       0.00000000e+00, 7.73860277e+01, 9.14620807e+01, 2.71319339e+02,
       1.14754340e+02, 1.06745759e+02, 8.66997511e+01, 1.10285836e+02,
       1.37791901e+02, 9.79185800e+01, 1.71490363e+02, 2.08035658e+02,
       1.17140514e+02, 1.91314596e+02, 9.07487562e+01, 3.71426186e+01,
       2.90221571e+02, 2.15562064e+01, 2.83093128e+02, 2.52930530e+01,
       9.50490064e+01, 1.13170481e+02, 1.40737465e+02, 1.09972952e+02,
       9.72556960e+01, 9.04350450e+01, 1.55909864e+02, 0.00000000e+00,
       9.37906658e+01, 4.54043655e+01, 1.60334633e+02, 7.43588963e+01,
       0.00000000e+00, 1.60849161e+02, 6.39648556e+01, 0.00000000e+00,
       2.55044144e+00, 6.11195316e+01, 0.00000000e+00, 2.71292986e+01,
       3.41003692e+01, 3.40153297e+01, 2.99034507e+01, 0.00000000e+00,
       2.67038101e+01, 5.81498187e+01, 7.26501274e+01, 8.29965720e+01,
       1.52154945e+02, 6.89710696e+01, 6.98177552e+01, 6.23324679e+01,
       1.16752352e+02, 6.55202594e+01, 8.22454137e+01, 9.72311934e+01,
       4.38219567e+01, 4.50377785e+01, 4.60103452e+01, 0.00000000e+00,
       3.88036349e+01, 1.52570887e+02, 1.90870878e+02, 7.56793957e+01,
       1.23233184e+02, 5.33006835e+01, 1.10664774e+02, 0.00000000e+00,
       8.78388717e+01, 3.48683785e+01, 2.80724115e+01, 3.74352793e+01,
       8.01722752e+01, 5.78409146e+01, 2.92307342e+01, 2.45125079e+01,
       2.07374692e+01, 2.84167104e+01, 7.00432397e+01, 1.70003038e+01,
       5.36862536e+01, 4.66982172e+01, 4.53272821e+01, 5.00275181e+01,
       3.34624193e+01, 2.28817925e+01, 5.43341092e+01, 1.34709235e+01,
       2.51156351e+01, 3.30048005e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00]), array([ 2.44874510e+02,  2.45561031e+02,  1.18064569e+02,  9.35145771e+01,
        1.56569063e+02,  2.84158571e+02,  1.69223122e+02,  9.47892758e+01,
        1.52724290e+02,  1.51661332e+02,  1.64642196e+02,  1.35852779e+02,
        1.02249780e+02,  2.00144784e+02,  1.59569962e+02,  1.69580725e+02,
        1.59098265e+02,  2.90374240e+02,  1.91347887e+02,  1.73700999e+02,
        1.50026693e+02,  1.56699471e+02,  1.57826570e+02,  3.16101983e+02,
        2.56514261e+02,  1.01132545e+02,  1.02528502e+02,  1.18471786e+02,
        1.06273142e+02,  8.61719252e+01,  1.48447834e+02,  3.32572376e+01,
        6.95614154e+01,  1.95600279e+01,  1.05414832e+02,  1.78483306e+02,
        1.17530085e+02,  1.37198945e+02,  6.63587029e+01,  1.65602825e+02,
        7.89110080e+01,  6.51292063e+01,  7.15896532e+01,  1.51903621e+02,
        1.20105254e+02,  2.83538272e+02,  1.45712663e+02,  1.79217735e+02,
        2.34424007e+02,  1.72077349e+02,  7.16372039e+01,  1.26728148e+02,
        8.30095582e+01,  1.42785074e+02,  1.53794231e+02,  1.41463334e+02,
        1.46732394e+02,  1.46212877e+02,  1.12795222e+02,  1.38759144e+02,
        1.17358832e+02,  2.01182027e+02,  2.45398660e+02,  8.45186988e+01,
        4.59515422e+01,  1.12804333e+02,  1.61895799e+02,  6.07140210e+01,
        1.16644132e+02,  4.62632228e+01,  5.62092016e+01,  4.66547102e+01,
        4.50998909e+01,  1.44257038e+02,  1.71565547e+02,  1.68302263e+02,
        7.52661283e+01,  5.29181998e+01,  2.27552242e+01,  2.53532196e+01,
        7.34470685e+01,  6.07876708e+01,  3.12185520e+01,  4.80172763e+01,
        8.92713746e+00, -1.04367320e+00,  9.99931290e+00,  3.27401586e+01,
        5.03664798e+01,  3.64735470e+01,  4.14848408e+01,  5.21841146e+01,
        5.18836986e+01,  5.87102182e-02,  1.91574382e+01,  5.21984299e+01,
        6.99509850e+01,  4.16805286e+01,  5.62048951e+01,  4.25729365e+01,
        5.29457807e+01,  7.75836013e+01,  3.23258948e+01,  4.48020151e+01,
        2.96265849e+01,  1.25992250e+02,  4.74628998e+01,  5.97853971e+01,
       -5.70609267e+00,  4.69146123e+01,  4.61299116e+01,  3.09313782e+01,
        3.24429813e+01,  1.78064953e+02,  2.32503485e+02,  1.11285751e+02,
        1.09556150e+02,  1.15893344e+02,  1.67431394e+02,  2.35548197e+02,
        9.12377483e+01,  1.10989781e+02,  5.25839535e+01,  4.86401883e+01,
        1.62322825e+01,  3.28321761e+01,  3.54082999e+01,  1.10899673e+01,
        3.45807663e+01,  3.41561075e+01,  4.48273438e+01,  2.74110565e+01,
        2.59420166e+01,  7.90942487e+01,  5.56982350e+01,  2.50453295e+01,
        3.21604254e+01,  3.50956267e+01,  8.20712267e+01,  1.45983276e+01,
        2.88124561e+01,  3.57050366e+01,  2.43244448e+01,  2.54497327e+01,
        2.92995432e+01,  2.75027163e+01,  6.41577565e+01,  6.88223263e+01,
        1.01299371e+02,  6.03876735e+01,  1.01839304e+02,  3.20305980e+01,
        1.90315933e+02,  6.12672856e+01,  4.98644326e+01,  8.61966493e+01,
        1.21138221e+02,  1.55794871e+02,  1.11546247e+02,  9.85778791e+01,
        9.49538228e+01,  1.36173043e+02,  4.83112437e+01,  7.60447793e+01,
        4.39367198e+01,  7.19092681e+01,  6.93524627e+01,  9.36575565e+01,
        6.30503143e+01,  5.31850453e+01,  3.22532541e+01,  3.85971418e+01,
        5.72416234e+01,  1.93627634e+01,  1.24852827e+01,  1.32754142e+01,
        5.43642513e+01,  1.75157096e+01,  3.11412684e+01,  2.79493135e+01,
        6.29833804e+01,  5.32903813e+01,  2.38684735e+00,  1.47420029e+01,
        1.35582253e+01,  4.15173003e+01,  1.57798758e+02,  8.26476395e+01,
        8.10577998e+01,  8.84402959e+01,  8.04457489e+01,  5.13965814e+01,
        8.16725674e+01,  9.07429791e+01,  1.18239956e+02,  6.00667304e+01,
        3.01884695e+00,  6.06533730e+01,  5.18298376e+01,  5.03813611e+01,
        5.79672952e+01,  1.73626711e+01,  2.65080447e+01, -5.18505764e+00,
       -1.57453545e+00,  1.46690073e+01,  9.17668205e+01,  5.59769034e+01,
        3.18362795e+01,  2.95477359e+01,  2.70224808e+01,  7.41347242e+01,
        1.08236790e+01,  5.97434750e+01,  8.06116572e+01,  1.73913830e+01,
        3.09970069e+01,  3.01662217e+01,  4.05652023e+01,  3.00411247e+01,
        2.85135348e+01,  3.02045586e+01,  2.14401013e+01,  1.46943013e+01,
        2.53201466e+01,  2.26413879e+01,  4.12041614e+01,  2.10981521e+01,
        2.98268913e+01,  1.43430853e+01,  1.01461951e+01,  1.33029340e+01,
        2.16358687e+02,  2.82745606e+02,  1.20740293e+02,  1.60074449e+02,
        2.98401372e+02,  1.76358754e+02,  1.11969920e+02,  1.94187576e+02,
        1.01287473e+02,  1.57557365e+02,  9.55201325e+01,  2.10921442e+02,
        1.71250093e+02,  2.28411053e+02,  1.88791856e+02,  2.69954032e+02,
        1.89324920e+02,  9.12866026e+01,  1.32433234e+02,  1.37148088e+02,
        7.88562907e+01,  1.60872792e+02,  1.43752626e+02,  1.46055539e+02,
        3.02180928e+02,  1.83431746e+02,  2.07430488e+02,  2.67572738e+02,
        2.95527311e+02,  2.58030965e+02,  8.42934081e+01,  8.32255205e+01,
        7.94051488e+01,  2.78932258e+02,  1.46007672e+02,  1.41900538e+02,
        7.91255078e+01,  7.84860537e+01,  6.95669829e+01,  1.22076569e+02,
        7.73520637e+01,  1.46333109e+02,  1.40657165e+02,  1.45506319e+02,
        1.29019175e+02,  1.39761354e+02,  2.40302707e+02,  2.28428115e+02,
        1.89612088e+02,  2.06909854e+02,  2.95604335e+02,  8.44357944e+01,
        8.38656925e+01,  7.99225631e+01,  1.58350599e+02,  9.67924269e+01,
        1.31747330e+02,  1.21339177e+02,  7.67482408e+01,  1.60314276e+02,
        1.31769949e+02,  1.45260784e+02,  1.53850408e+02,  1.38601107e+02,
        2.03894857e+02,  4.99271110e+02,  4.50666248e+02,  4.91324103e+02,
        1.26252049e+02,  1.37965319e+02,  1.24150267e+02,  1.21948303e+02,
        1.32495703e+02,  1.32725114e+02,  1.23290092e+02,  1.19813279e+02,
        1.52116412e+02,  1.19746741e+02,  1.29140476e+02,  1.24422799e+02,
        1.12846048e+02,  1.29632485e+02,  9.27127668e+01,  3.83894916e+01,
        4.18342037e+01,  9.66647494e+01,  1.25250341e+02,  5.73615873e+01,
        6.91458359e+01,  8.16194493e+01,  9.21864809e+01,  3.78907768e+01,
        1.05774203e+02,  5.77689866e+01,  2.50867958e+02,  1.25658094e+02,
        1.75247455e+02,  1.79741111e+02,  1.19796507e+02,  1.23016849e+02,
        1.66456846e+02,  1.21819011e+02,  1.88612299e+02,  1.04577028e+02,
        7.84340589e+01,  1.31127678e+02,  1.24543922e+02,  2.95331112e+02,
        1.51244929e+02,  1.96139639e+02,  1.08223005e+02,  1.46022769e+02,
        1.41969147e+02,  1.15388715e+02,  1.14278463e+02,  1.47317984e+02,
        1.10950243e+02,  1.53462896e+02,  1.19898009e+02,  5.47067981e+01,
        8.07685255e+01,  4.45120867e+01,  1.11783163e+02,  2.86869898e+01,
        5.19625873e+01,  1.23251625e+02,  9.13617971e+01,  8.23332753e+01,
        9.70987867e+01,  1.29320282e+02,  4.60444875e+01,  4.37219639e+01,
        8.48331214e+01,  4.51970315e+01,  6.63130485e+01,  5.69853357e+01,
        3.72029713e+01,  7.41500181e+01,  5.49797190e+01,  3.05117379e+00,
        4.97442550e+00,  3.83264459e+01,  2.87359313e+01,  2.76017530e+01,
        4.07976987e+01,  4.36303749e+01,  3.75909036e+01,  2.18081119e+01,
        3.16535934e+01,  5.34047603e+01,  4.57472847e+01,  8.01795198e+01,
        1.03976630e+02,  5.43249107e+01,  7.15468707e+01,  3.45983561e+01,
        4.50665563e+01,  5.54773453e+01,  6.65056344e+01,  9.72936560e+01,
        2.93304149e+01,  3.39537347e+01,  2.53390403e+01,  2.87911941e+01,
        2.90899007e+01,  1.61732670e+02,  2.43458247e+02,  1.02790328e+02,
        1.45065289e+02,  7.31774377e+01,  8.42018421e+01,  4.78416322e+01,
        8.84964886e+01,  1.55949825e+01,  1.67783084e+01,  1.59700589e+01,
        5.82226984e+01,  5.54631297e+01,  1.84212120e+01,  3.36016107e+01,
        5.71471065e+01,  2.26345392e+01,  4.00741108e+01,  2.70303170e+01,
        3.62032652e+01,  2.99579861e+01,  2.75004456e+01,  3.68492909e+01,
        2.87704715e+01,  3.21158637e+01,  3.40938563e+01,  5.07441054e+01,
        3.25449381e+01,  1.33678242e+01,  8.25227827e+00,  3.02201201e+01,
       -1.13066380e+00]), OrderedDict([('R2', 0.6711371482150259), ('root_mean_squared_error', 60.08917807746408), ('mean_absolute_error', 37.5592367999205), ('rmse_over_stdev', 0.64363188553948)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/StandardScaler/SequentialFeatureSelector/KernelRidge/RepeatedKFold/split_3'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
